In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())


100%|██████████| 9.91M/9.91M [00:00<00:00, 54.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.72MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.28MB/s]


# Data loader

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


# Loss and optimizer

In [8]:
model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model


In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.2533
Epoch [1/5], Step [200/600], Loss: 0.1046
Epoch [1/5], Step [300/600], Loss: 0.0681
Epoch [1/5], Step [400/600], Loss: 0.0326
Epoch [1/5], Step [500/600], Loss: 0.0662
Epoch [1/5], Step [600/600], Loss: 0.0753
Epoch [2/5], Step [100/600], Loss: 0.0784
Epoch [2/5], Step [200/600], Loss: 0.0656
Epoch [2/5], Step [300/600], Loss: 0.0333
Epoch [2/5], Step [400/600], Loss: 0.0289
Epoch [2/5], Step [500/600], Loss: 0.0387
Epoch [2/5], Step [600/600], Loss: 0.0389
Epoch [3/5], Step [100/600], Loss: 0.1201
Epoch [3/5], Step [200/600], Loss: 0.0708
Epoch [3/5], Step [300/600], Loss: 0.0103
Epoch [3/5], Step [400/600], Loss: 0.0252
Epoch [3/5], Step [500/600], Loss: 0.0336
Epoch [3/5], Step [600/600], Loss: 0.0583
Epoch [4/5], Step [100/600], Loss: 0.0131
Epoch [4/5], Step [200/600], Loss: 0.0368
Epoch [4/5], Step [300/600], Loss: 0.0208
Epoch [4/5], Step [400/600], Loss: 0.0239
Epoch [4/5], Step [500/600], Loss: 0.0678
Epoch [4/5], Step [600/600], Loss:

# Test the model

In [11]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.99 %
